In [244]:
#Import relevant libraries
import sklearn
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from keras.wrappers import scikit_learn
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from collections import Counter, defaultdict
from itertools import combinations 
import operator
import math
import itertools
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing, tree
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [255]:
#Load data
df_train = pd.read_csv('dma2022-competition/train.csv')
df_test = pd.read_csv('dma2022-competition/test.csv')

In [256]:
#One hot encode 'Sex'
ohe = OneHotEncoder()
transformed = ohe.fit_transform(df_train[['Sex']]).toarray()
df_train['Sex'] = transformed.astype(int)

#Normalize 'Fare'
df_train['Fare'] = df_train['Fare'].fillna(0)
df_train['Fare'] = (df_train['Fare'] - df_train['Fare'].mean()) / df_train['Fare'].std()

#New 'Fare_Category' feature
df_train['Fare_Category'] = pd.cut(df_train['Fare'], bins=[min(df_train['Fare'])-1,max(df_train['Fare'])*0.66,max(df_train['Fare'])], labels=[0,1])

#'Embarked' dummies
one_hot_embarked = pd.get_dummies(df_train['Embarked'], prefix='embarked')
df_train = df_train.drop('Embarked',axis = 1)
df_train= df_train.join(one_hot_embarked)

#New 'Is_Alone' feature
df_train['Family'] = df_train.Parch + df_train.SibSp
df_train['Is_Alone'] = (df_train.Family == 0).astype(int)

#New 'age_group' feature
def age_group(series):
    if series['Age'] <=15:
        return 'Children'
    elif (series['Age'] >15) and (series['Age']<65):
        return 'Adult'
    else:
        return 'Elder'

df_train['age_group'] = df_train.apply(age_group,axis=1)
one_hot_age_group = pd.get_dummies(df_train['age_group'])
df_train = df_train.drop('age_group',axis = 1)
df_train = df_train.join(one_hot_age_group)

#Normalize 'Age'
df_train['Age'] = df_train['Age'].fillna(0)
df_train['Age'] = (df_train['Age'] - df_train['Age'].mean()) / df_train['Age'].std()

#New 'Title' feature
df_train['Title'] = df_train['Name'].astype(str).str.split().str[1]
one_hot_title = pd.get_dummies(df_train['Title'], prefix='title')
df_train = df_train.drop('Title',axis = 1)
df_train= df_train.join(one_hot_title)

df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,"title_Pelsmaeker,","title_Planke,",title_Rev.,"title_Shawah,","title_Steen,","title_Velde,","title_Walle,",title_der,title_the,title_y
0,1,0,3,"Braund, Mr. Owen Harris",0,-0.102255,1,0,A/5 21171,-0.502163,...,0,0,0,0,0,0,0,0,0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,0.807038,1,0,PC 17599,0.786404,...,0,0,0,0,0,0,0,0,0,0
2,3,1,3,"Heikkinen, Miss. Laina",1,0.125068,0,0,STON/O2. 3101282,-0.488580,...,0,0,0,0,0,0,0,0,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,0.636546,1,0,113803,0.420494,...,0,0,0,0,0,0,0,0,0,0
4,5,0,3,"Allen, Mr. William Henry",0,0.636546,0,0,373450,-0.486064,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,0.181899,0,0,211536,-0.386454,...,0,0,1,0,0,0,0,0,0,0
887,888,1,1,"Graham, Miss. Margaret Edith",1,-0.272748,0,0,112053,-0.044356,...,0,0,0,0,0,0,0,0,0,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,-1.352534,1,2,W./C. 6607,-0.176164,...,0,0,0,0,0,0,0,0,0,0
889,890,1,1,"Behr, Mr. Karl Howell",0,0.125068,0,0,111369,-0.044356,...,0,0,0,0,0,0,0,0,0,0


In [257]:
#Split test and train data
X_train, y_train = df_train[['Sex', 'Pclass','Children']], df_train[['Survived']]
#'Sex', 'Pclass', 'Fare', 'Is_Alone', 'embarked_C', 'title_Dr.', 'title_Master.', 'Children', 'Elder', 'Age'

In [258]:
#Models
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
qda_clf = MLPClassifier(hidden_layer_sizes=(15), activation = 'logistic', solver ='lbfgs', random_state = 42)

#Keras Model
def build_nn():
    model= Sequential([
                       Dense(50,activation='relu',input_shape=[len(X_train.columns)]),
                       Dense(1,activation='sigmoid')
])
    model.compile(
                optimizer='Adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return model

keras_clf = scikit_learn.KerasClassifier(
                            build_nn,
                            epochs=500,
                            verbose=False)

keras_clf._estimator_type = "classifier"

/tmp/ipykernel_112/2605251477.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_clf = scikit_learn.KerasClassifier(


In [249]:
#Voting Classifier
Voting = VotingClassifier(
             estimators=[('lr', log_clf),
                         ('rf', rnd_clf),
                         ('keras',keras_clf),
                         ('qda', qda_clf)], 
             voting='soft',
             flatten_transform=True)

Voting.fit(X_train,y_train)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('keras',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f5893a26d30>),
                             ('qda',
                              MLPClassifier(activation='logistic',
                                            hidden_layer_sizes=15,
                                            random_state=42, solver='lbfgs'))],
                 voting='soft')

In [240]:
#Accuracies
accuracies = {}
for clf in (log_clf, rnd_clf, keras_clf, qda_clf, Voting):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_train)
    print(clf.__class__.__name__, accuracy_score(y_train, y_pred))
    accuracies[clf.__class__.__name__] = accuracy_score(y_train, y_pred)
accuracies

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/tmp/ipykernel_112/1959225379.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)
LogisticRegression 0.7856341189674523
RandomForestClassifier 0.8069584736251403
28/28 [==============================] - 0s 4ms/step
KerasClassifier 0.8069584736251403
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/shared-libs/python3.9/py/lib/python3.

{'LogisticRegression': 0.7856341189674523,
 'RandomForestClassifier': 0.8069584736251403,
 'KerasClassifier': 0.8069584736251403,
 'MLPClassifier': 0.8069584736251403,
 'VotingClassifier': 0.8069584736251403}

In [ ]:
# Sex, Pclass, embarked_ C 'VotingClassifier': 0.7867564534231201
# Sex, Pclass, Is_Alone  'VotingClassifier': 0.8024691358024691
# Sex, Pclass, embarked_C, Is_Alone

In [259]:
#Test Data
ohe = OneHotEncoder()
transformed = ohe.fit_transform(df_test[['Sex']]).toarray()
df_test['Sex'] = transformed.astype(int)

#Normalize 'Fare'
df_test['Fare'] = df_test['Fare'].fillna(0)
df_test['Fare'] = (df_test['Fare'] - df_test['Fare'].mean()) / df_test['Fare'].std()

#'Embarked' dummies
one_hot_embarked = pd.get_dummies(df_test['Embarked'], prefix='embarked')
df_test = df_test.drop('Embarked',axis = 1)
df_test= df_test.join(one_hot_embarked)

#'Title' feature
df_test['Title'] = df_test['Name'].astype(str).str.split().str[1]
one_hot_title = pd.get_dummies(df_test['Title'], prefix='title')
df_test = df_test.drop('Title',axis = 1)
df_test= df_test.join(one_hot_title)

#'Is_Alone' Feature
df_test['Family'] = df_test.Parch + df_test.SibSp
df_test['Is_Alone'] = (df_test.Family == 0).astype(int)

#'Fare_Category' feature
df_test['Fare_Category'] = pd.cut(df_test['Fare'], bins=[min(df_test['Fare'])-1,max(df_test['Fare'])*0.66,max(df_test['Fare'])], labels=[0,1])

#'age_group' feature
df_test['age_group'] = df_test.apply(age_group,axis=1)
one_hot_age_group = pd.get_dummies(df_test['age_group'])
df_test = df_test.drop('age_group',axis = 1)
df_test = df_test.join(one_hot_age_group)

#Normalize 'Age'
df_test['Age'] = df_test['Age'].fillna(0)
df_test['Age'] = (df_test['Age'] - df_test['Age'].mean()) / df_test['Age'].std()

In [260]:
qda_clf.fit(X_train,y_train)
#Predictions
X_test = df_test[['Sex', 'Pclass','Children']]
y_pred_test = qda_clf.predict(X_test)

#Make prediction df
prediction_df = df_test[['PassengerId']]
prediction_df['Survived'] = y_pred_test
prediction_df

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/tmp/ipykernel_112/771352458.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction_df['Survived'] = y_pred_test


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [262]:
#Save prediction df
prediction_df.to_csv(r'prediction_kaggle_competition_Ensemble_Final_4.csv', index = False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ac8a932a-48c3-4492-b3a4-cefdce18987a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>